In [33]:
import urllib.request, urllib.error, urllib.parse
import requests
import csv
from urllib.request import build_opener, HTTPCookieProcessor, Request

#PUT JOURNAL URL HERE
url = 'https://www.tandfonline.com/loi/wjsa21'


opener = build_opener(HTTPCookieProcessor())
request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
response = opener.open(request, timeout=30)
allContent = str(response.read())

In [34]:
def toSheetHeader(name):
    # field names  
    fields = ['Title', 'Abstract','Journal']  
    
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'w') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the fields  
        csvwriter.writerow(fields)  

In [35]:
def toSheet(rows,name):
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'a') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the data rows  
        csvwriter.writerows(rows) 

In [36]:
allarticles = []


#GETTING JOURNAL TITLE


journal_index = allContent.find('<title')
journal_index = allContent.find('>',journal_index) + 16
journal = ""
while(allContent[journal_index]!="<"):
    journal = journal + allContent[journal_index]
    journal_index = journal_index+1


journal = journal.replace("\\n"," ")

toSheetHeader(journal)
                    
#FINDING THE LINK TO THE ISSUE, allContent is all of the code for the original URL                    
issue_index = 0

while(allContent.find('<div class="volume_link">',issue_index,len(allContent))!=-1 and 
      allContent.find('2015',allContent.find('<div class="volume_link">',issue_index,len(allContent)),len(allContent))!=-1):
    
    issue_index = allContent.find('<div class="volume_link">',issue_index,len(allContent))
    i = issue_index-100
    i = allContent.find('href=', i, len(allContent)) + 6
    
    issue_link = ""
                                  
    while(allContent[i]!='"'):
        issue_link = issue_link+allContent[i]
        i = i+1
    
    #getting code for ISSUE LINK here
    opener = build_opener(HTTPCookieProcessor())
    request = Request("https://www.tandfonline.com"+issue_link, headers={'User-Agent': 'Mozilla/5.0'})
    response = opener.open(request, timeout=30)
    web1Content = str(response.read())

    
    #finding issue
    curr_index = 0
    while(web1Content.find('<a class="issue-link"',curr_index,len(web1Content))!=-1):
        curr_index = web1Content.find('<a class="issue-link"',curr_index,len(web1Content))
        
        iss_link_index = web1Content.find("href=",curr_index,len(web1Content))
        iss_link = ""
        i = iss_link_index+6
        while(web1Content[i]!='"'):
            iss_link = iss_link+web1Content[i]
            i = i+1
           
        
        opener = build_opener(HTTPCookieProcessor())
        request = Request("https://www.tandfonline.com"+iss_link, headers={'User-Agent': 'Mozilla/5.0'})
        response = opener.open(request, timeout=30)
        webContent = str(response.read())
    
    
        article_index = 0  
        ind = 0
        while(webContent.find('class="art_title linkable',article_index,len(webContent))!=-1):
            currarticle = []
            article_index = webContent.find('class="art_title linkable',article_index,len(webContent))
        
            link_index = webContent.find("href=",article_index,len(webContent))
            link = ""
            
            i = link_index+6
            while(webContent[i]!='"'):
                link = link+webContent[i]
                i = i+1

            
            title = ""
      
            i = webContent.find('class="hlFld-Title">',i,len(webContent))
            i = i + 20
            
            while(webContent[i]!='<'):
                title = title+webContent[i]
                i = i+1

            title = title.replace("\\n","")
            currarticle.append(title)
            #print("\033[1m",title,"\033[0;0m")

        
      
            #OPENING LINK TO ARTICLE
            opener = build_opener(HTTPCookieProcessor())
            request = Request("https://www.tandfonline.com"+link, headers={'User-Agent': 'Mozilla/5.0'})
            response = opener.open(request, timeout=30)
            articleContent = str(response.read())
            print("https://www.tandfonline.com"+link)
            
            
        
            execute = True
            abstract_index = articleContent.find('<p class="summary-title">') 
           
            if (abstract_index == -1):
                abstract_index = articleContent.find('abstractInFull"')
            if (abstract_index == -1):
                abstract_index = articleContent.find('<div class="section1')
            if (abstract_index == -1):
                execute = False
            print(abstract_index)
            
            if(execute):    
                abstract_index = articleContent.find("<p",abstract_index+1,len(articleContent))
                abstract_index = articleContent.find(">",abstract_index,len(articleContent))
                i = abstract_index+1
      
                end_abstract_index = articleContent.find("</p>",abstract_index,len(articleContent))

                abstract = ""

                while(i!=end_abstract_index):
                    abstract = abstract+articleContent[i]
                    i = i + 1
    
                abstract = abstract.replace("\\n"," ")
                currarticle.append(abstract)

            currarticle.append(journal)
            allarticles.append(currarticle)
            toSheet(allarticles,journal)
            allarticles=[]
            article_index = article_index+1
            issue_index = issue_index+1
            curr_index = curr_index +1


https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1791489
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1775752
38756
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1766635
36594
https://www.tandfonline.com/doi/full/10.1080/21683565.2019.1646374
38132
https://www.tandfonline.com/doi/full/10.1080/21683565.2019.1680476
40785
https://www.tandfonline.com/doi/full/10.1080/21683565.2019.1711288
39149
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1752350
36291
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1774110
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1719450
37462
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1726550
38076
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1725216
39951
https://www.tandfonline.com/doi/full/10.1080/21683565.2019.1631935
38374
https://www.tandfonline.com/doi/full/10.1080/21683565.2020.1712572
39367
https://www.tandfonline.com/doi/full/10.1080/21683565.202

https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1532480
38423
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1547941
37303
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1515800
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1529358
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1509410
38529
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1509168
38017
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1509167
39959
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1485126
37549
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1537323
39267
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1515541
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1516711
37113
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1537324
37636
https://www.tandfonline.com/doi/full/10.1080/21683565.2019.1566192
36924
https://www.tandfonline.com/doi/full/10.1080/21683565.2018.1

https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1331179
40127
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1326073
36848
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1283575
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1308898
36999
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1320349
36964
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1320620
39252
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1290731
36681
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1310785
37668
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1310786
38044
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1319891
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1313674
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1279252
37397
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1279703
39450
https://www.tandfonline.com/doi/full/10.1080/21683565.2017.1

https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1071722
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1053586
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1067940
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1069776
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1069775
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1073207
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1052125
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1050147
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1046537
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1050148
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1050146
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1053587
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1039159
-1
https://www.tandfonline.com/doi/full/10.1080/21683565.2015.1020406
-1
https://www.tandfonl

KeyboardInterrupt: 